In [363]:
from merge_packages import merge_packages

import numpy as np
import pandas as pd

import json
import os

import requests
from bs4 import BeautifulSoup

from tqdm.auto import tqdm


In [364]:
cur_packages = merge_packages()

with open('data/homepages.json') as cur_file:
    cur_homepages = json.load(cur_file)
    
with open('data/anchors.json') as cur_file:
    cur_anchors = json.load(cur_file)

In [365]:
cur_branches = ["/stable", "/latest", "/dev"]
cur_protocols = ["https://", "http://"]

In [366]:
cur_docs = []

for cur_row in cur_packages.itertuples():
    if cur_row.name in cur_homepages and cur_homepages[cur_row.name]:
        cur_docs.append(cur_homepages[cur_row.name])
        continue
        
    if cur_row.name not in cur_anchors or len(cur_anchors[cur_row.name]) == 0:
        cur_docs.append(np.nan)
        continue
        
    sub_anchors = cur_anchors[cur_row.name]

    if len(sub_anchors) == 1:
        cur_docs.append(sub_anchors[0])
        continue
        
    work_anchors = []
    
    for sub_anchor in sub_anchors:
        for cur_branch in cur_branches:
            if not sub_anchor.endswith(cur_branch) : continue
            short_link = "/".join(sub_anchor.split("/")[:-1]).lower()
            
            for cur_protocol in cur_protocols:
                if cur_protocol not in short_link : continue
                short_link = short_link.split("://")[-1]
            
            if short_link in work_anchors : continue
            work_anchors.append(short_link)
            
    if len(work_anchors) > 1:
        work_anchors = [
            work_anchor for work_anchor in work_anchors
            if cur_row.name.lower() in work_anchor.lower()
        ]
            
    if len(work_anchors) == 1:
        did_add = False
        
        for cur_branch in cur_branches:
            medium_link = f"{work_anchors[0]}{cur_branch}"
            
            for cur_protocol in cur_protocols:
                long_link = f"{cur_protocol}{medium_link}"
                
                for sub_anchor in sub_anchors:
                    if long_link != sub_anchor.lower() : continue
                    cur_docs.append(sub_anchor)
                    
                    did_add = True
                    break
                    
                if did_add : break
                    
            if did_add : break
                
        assert did_add
        continue
    
    work_anchors = []
    
    for sub_anchor in sub_anchors : 
        if f"github.io/{ cur_row.name.lower() }" not in sub_anchor.lower() : continue
        work_anchors.append(sub_anchor)
       
    if len(work_anchors) == 1 :
        cur_docs.append(work_anchors[0])
        continue
        
    work_anchors = []
    
    for sub_anchor in sub_anchors : 
        if "doc" not in sub_anchor.lower() : continue
        work_anchors.append(sub_anchor)
        
    if len(work_anchors) > 1 :
        work_anchors = [
            work_anchor for work_anchor in work_anchors
            if cur_row.name.lower() in work_anchor.lower()
        ]
        
    if len(work_anchors) == 1 :
        cur_docs.append(work_anchors[0])
        continue
        
    work_anchors = []
    
    for sub_anchor in sub_anchors : 
        if cur_row.name.lower() not in sub_anchor.lower() : continue
        work_anchors.append(sub_anchor)
        
    if len(work_anchors) == 1 :
        cur_docs.append(work_anchors[0])
        continue
        
    work_anchors = []
    
    for sub_anchor in sub_anchors : 
        if "readthedocs" not in sub_anchor.lower() : continue
        if cur_row.name.lower() not in sub_anchor.lower() : continue
        work_anchors.append(sub_anchor)
        
    if len(work_anchors) > 0 :
        cur_docs.append(work_anchors[0])
        continue
        
    work_anchors = []
    
    for sub_anchor in sub_anchors : 
        if "github.io" not in sub_anchor.lower() : continue
        if cur_row.name.lower() not in sub_anchor.lower() : continue
        work_anchors.append(sub_anchor)
        
    if len(work_anchors) > 0 :
        cur_docs.append(work_anchors[0])
        continue
        
    cur_docs.append(np.nan)
    
assert len(cur_docs) == len(cur_packages)


In [391]:
for (cur_index, cur_doc) in tqdm(enumerate(cur_docs), total=len(cur_docs)):    
    cur_package = cur_packages.name.iloc[cur_index]
    
    cur_name = f"tmp/docs/{cur_package}.txt"
    if os.path.exists(cur_name) : continue
        
    print(cur_package, cur_doc)
        
    if type(cur_doc) != str and np.isnan(cur_doc):
        with open(cur_name, "w") as cur_file:
            cur_file.write("")
        continue
        
    cur_request = None
    try:
        cur_request = requests.get(cur_doc, timeout=20)
    except requests.RequestException:
        print(cur_package)
        continue
    assert cur_request != None

    cur_soup = BeautifulSoup(cur_request.text, 'html.parser')
    
    cur_items = cur_soup(['style', 'script', '[document]', 'head', 'title'])
        
    cur_items.extend(cur_soup.select('[style~="visibility:hidden"]'))
    cur_items.extend(cur_soup.select('[style~="display:none"]'))
    cur_items.extend(cur_soup.select('.hidden'))
    
    for tmp_soup in cur_items:
        tmp_soup.extract()
      
    visible_text = cur_soup.get_text(separator=' ')
    visible_text = re.sub(r"\s\s+", " ", visible_text)
    visible_text = re.sub(r"\n\n+", "\n", visible_text)
    
    with open(cur_name, "w") as cur_file:
        cur_file.write(visible_text)


Compose3D http://composejl.org
Compose3D
ComposeDiff http://composejl.org
ComposeDiff
CRlibm http://lipforge.ens-lyon.fr/www/crlibm
CRlibm
CUDAnativelib https://ci.maleadt.net/buildbot/julia/builders/CUDAnativelib.jl%3A%20Julia%20master%20%28x86-64%29
CUDAnativelib
CUFFT http://ci.maleadt.net/shields/url.php
CUFFT
FunHPC https://dependencyci.com/github/eschnett/FunHPC.jl
FunHPC
GenericSchur http://www.netlib.org/lapack/index.html
GenericSchur
GLVisualize https://ci.maleadt.net/buildbot/julia/builders/GLVisualize.jl%3A%20Julia%200.6%20%28x86-64%29
GLVisualize
Hydna https://www.hydna.com
Hydna
ICOADSDict http://rda.ucar.edu/datasets/ds540.0/docs/ICOADS2.5-HD_Brohan2015.mp4


KeyboardInterrupt: 